Multimodel Architecture - Routing Workflow 

In [1]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [2]:
# Always remember to do this!
load_dotenv(override=True)

True

In [3]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [4]:
openai_client = OpenAI(api_key=openai_api_key)
deepseek_client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
gemini_client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
groq_client = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")

In [5]:
MODEL_REGISTRY = {
    "gpt-5-nano": {
        "provider": "openai",
        "strength": "general",
        "cost": "low"
    },
    "gpt-5-mini": {
        "provider": "openai",
        "strength": "reasoning",
        "cost": "medium"
    },
    
    "deepseek-chat": {
        "provider": "deepseek",
        "strength": "coding",
        "cost": "low"
    },
    "gemini-2.5-flash": {
        "provider": "google",
        "strength": "general",
        "cost": "low"
    }
}


ROUTER AGENT

In [ ]:
def classify_task(user_input):
    router_prompt = f"""
    Classify the task into ONE of these categories:
    - coding
    - creative_writing
    - quantitative_reasoning
    - strategic_analysis
    - simple_general

    Respond ONLY with the category name.

    User request:
    {user_input}
    """

    response = openai_client.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "user", "content": router_prompt}],
    )

    return response.choices[0].message.content.strip()


ROUTING LOGIC

In [12]:
def select_model(task_type):
    if task_type == "coding":
        return "deepseek-chat"
    elif task_type == "creative_writing":
        return "gemini-2.5-flash"
    elif task_type == "quantitative_reasoning":
        return "gpt-5-mini"
    elif task_type == "strategic_analysis":
        return "gpt-5-mini"
    else:
        return "gpt-5-nano"


EXECUTION LAYER

In [8]:
def call_model(model_name, user_input):

    if model_name in ["gpt-5-nano", "gpt-5-mini"]:
        response = openai_client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": user_input}],
        )
        return response.choices[0].message.content

    elif model_name == "deepseek-chat":
        response = deepseek_client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": user_input}],
        )
        return response.choices[0].message.content

    elif model_name == "gemini-2.5-flash":
        response = gemini_client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": user_input}],
        )
        return response.choices[0].message.content

    else:
        raise ValueError("Unknown model")


In [9]:
def route_and_execute(user_input):

    print("Classifying task...")
    task_type = classify_task(user_input)
    print("Task type:", task_type)

    model_name = select_model(task_type)
    print("Selected model:", model_name)

    print("Executing")
    answer = call_model(model_name, user_input)

    return answer


TESTING

In [10]:
user_question = "Write a Python function to calculate factorial recursively."

result = route_and_execute(user_question)

print("\nFinal Answer:\n")
print(result)


Classifying task...
Task type: coding
Selected model: deepseek-chat
Executing

Final Answer:

Here's a Python function to calculate factorial recursively:

```python
def factorial(n):
    """
    Calculate the factorial of a non-negative integer n recursively.
    
    Parameters:
    n (int): A non-negative integer
    
    Returns:
    int: The factorial of n (n!)
    
    Raises:
    ValueError: If n is negative
    """
    # Base case: factorial of 0 is 1
    if n == 0:
        return 1
    
    # Error case: factorial is not defined for negative numbers
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    
    # Recursive case: n! = n * (n-1)!
    return n * factorial(n - 1)


# Example usage
if __name__ == "__main__":
    # Test cases
    test_numbers = [0, 1, 5, 7, 10]
    
    for num in test_numbers:
        result = factorial(num)
        print(f"factorial({num}) = {result}")
    
    # Test with negative number (will raise ValueError)
 

In [11]:
user_question = "If a company reduces prices by 12% and sales volume increases by 18%, under what conditions does total revenue increase?"

result = route_and_execute(user_question)

print("\nFinal Answer:\n")
print(result)

Classifying task...
Task type: quantitative_reasoning
Selected model: gpt-5-mini
Executing

Final Answer:

Let p = 12% be the price cut and q = 18% the increase in quantity.

New revenue / old revenue = (1 − p)(1 + q) = 0.88 × 1.18 = 1.0384, so revenue rises by 3.84%.

In general revenue increases iff (1 − p)(1 + q) > 1, i.e.
q > p/(1 − p).

For p = 0.12 this threshold is 0.12/0.88 ≈ 13.636%. Since q = 18% > 13.636%, revenue increases.

Equivalently (using elasticity): the price cut raises revenue when demand is elastic (|%ΔQ| / |%ΔP| > 1). Here 18%/12% = 1.5 > 1, so revenue increases.


In [13]:
user_question = "Write a 150 word essay on AI impacting creative industry"

result = route_and_execute(user_question)

print("\nFinal Answer:\n")
print(result)

Classifying task...
Task type: creative_writing
Selected model: gemini-2.5-flash
Executing

Final Answer:

AI is deeply reshaping the creative industry, offering immense opportunities and complex challenges. It empowers artists, designers, and writers by accelerating idea generation, streamlining workflows, and enabling rapid prototyping. AI tools democratize creation, lowering barriers for individuals to produce high-quality content, from personalized marketing to novel art forms, boosting efficiency and expanding artistic boundaries.

However, job displacement concerns are valid as AI automates routine tasks. Ethical dilemmas surrounding copyright, data ownership, and fair artist compensation are pressing. The debate about AI-generated art's authenticity versus human creations also persists. Ultimately, the creative industry must adapt, viewing AI as a powerful co-pilot and tool, not a full replacement, to navigate this evolving future.


In [14]:
user_question = "A mid-sized consulting firm is losing clients due to AI automation. Propose a structured turnaround strategy."

result = route_and_execute(user_question)

print("\nFinal Answer:\n")
print(result)

Classifying task...
Task type: strategic_analysis
Selected model: gpt-5-mini
Executing

Final Answer:

Short summary
- Problem: clients are switching to AI automation because the firm’s services are increasingly commoditized, price-sensitive, and easily replicated by AI tools.
- Goal: reposition the firm from pure labor-driven delivery to a higher-value, AI-augmented advisor that combines domain expertise, outcome guarantees, and turnkey AI-enabled products and managed services — while cutting costs, stabilizing cash flow, and rebuilding growth.

High-level turnaround objectives (12 months)
1. Stop client churn and stabilize revenue within 90 days.
2. Launch 2–3 AI-augmented productized offerings and 1 managed-service capability within 6 months.
3. Re-skill core consulting staff and build an AI/analytics capability to support scaled delivery within 12 months.
4. Achieve positive EBITDA improvement and restore pipeline velocity by month 12.

Quick diagnosis — why clients leave
- Client 

In [15]:
user_question = "A mid-sized consulting firm is losing clients due to AI automation. Propose a structured turnaround strategy."

result = route_and_execute(user_question)

print("\nFinal Answer:\n")
print(result)

Classifying task...
Task type: strategic_analysis
Selected model: gpt-5-mini
Executing

Final Answer:

Executive summary
- Problem: Clients are switching to AI automation (tools/products) that replace parts of traditional consulting work. That erodes revenue, pipeline and perceived relevance.
- Goal: Stabilize revenue, stop client churn, and rebuild a differentiated, sustainable services/offering model where the firm adds value that AI alone cannot.
- Approach: Three-phase turnaround (Stabilize 0–3 months; Rebuild 3–12 months; Grow & Scale 12–36 months) built on five strategic pillars: client centricity & retention, productized outcome offerings, AI-native advisory + delivery, talent & operating model, and partnerships & IP.

Diagnosis (what to check immediately)
- Client churn analysis: Which segments, services, and clients left and why (cost, speed, perceived equivalence of AI)?
- Service mapping: Which consulting tasks are commoditized by AI vs. which remain high-value (strategy, le